In [2]:
import pandas as pd
import string
import numpy as np
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from scipy import stats
import matplotlib
import math

/Users/denis/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Загрузка данных

In [3]:
def fix_columns(df):
    cols=list(df.columns)
    real_columns = ['COMPARABLE RENTAL 1 ']#[u'Boro-Block-Lot',u'Condo Section',u'Address',u'Neighborhood',u'Building Classification',\
                    #u'Total Units',u'Year Built',u'Gross SqFt',u'Est. Gross Income',u'Gross Income per SqFt',\
                    #u'Full Market Value',u'Market Value per SqFt',u'Boro-Block-Lot', u'Address',u'Neighborhood',\
                    #u'Building Classification',u'Total Units', u'Year Built',u'Gross SqFt',u'Est. Gross Income',\
                    #u'Gross Income per SqFt',u'Full Market Value',u'Market Value per SqFt',u'Dist. from Coop in miles',\
                    #u'Boro-Block-Lot', u'COMPARABLE RENTAL 2  Address',u'Neighborhood',u'Building Classification',\
                    #u'Total Units', u'COMPARABLE RENTAL 2  Year Built',u'Gross SqFt',u'Est. Gross Income',\
                    #u'Gross Income per SqFt',u'Full Market Value',u'Market Value per SqFt',u'Dist. from Coop in miles']
    to_remove = ['QUEENS CONDOMINIUM PROPERTY ', 'BRONX CONDOMINIUM PROPERTY ', 'BROOKLYN CONDOMINIUM PROPERTY ',\
                 'MANHATTAN CONDOMINIUM PROPERTY ', 'STATEN ISLAND CONDOMINIUM PROPERTY ',\
                'QUEENS CONDOMINIUMS COMPARABLE PROPERTIES ',\
                 'BRONX CONDOMINIUMS COMPARABLE PROPERTIES ','BROOKLYN CONDOMINIUMS COMPARABLE PROPERTIES ',\
                 'MANHATTAN CONDOMINIUMS COMPARABLE PROPERTIES ', 'STATEN ISLAND CONDOMINIUMS COMPARABLE PROPERTIES ']
    rental = 'COMPARABLE RENTAL '
    rm = []
    table = {}
    for c in cols:
        c_new = ' '.join(filter(lambda s: s!= '\xc3\xa2\xc2\x80\xc2\x93', c.split()))
        flag = True
        if flag:
            if c_new.startswith('COMPARABLE RENTAL 3'):
                rm.append(c)
        if flag:
            if c_new.startswith(rental):
                table[c] = ('RENTAL ' + c_new[len(rental):])
                flag = not flag
        if flag:
            for prefix in to_remove:
                if c_new.startswith(prefix):
                    table[c] = ('PROPERTY ' + c_new[len(prefix):])
                    flag = not flag
                    break
        if flag:    
            table[c] = c_new
    df = df.drop(rm, axis=1)
    df = df.rename(index=str, columns=table)
    return df

In [4]:
df = pd.DataFrame()
for i in range(2008, 2012):
    for j in ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten_Island']:
        new_df = pd.read_csv('Datasets/Condominium/%i_%i/DOF__Condominium_Comparable_Rental_Income___%s___FY_%i_%i.csv'\
        % (i, i+1, j, i, i+1))
        new_df['Year'] = i
        new_df['District'] = j
        df = pd.concat([df, fix_columns(new_df)], ignore_index=True)

In [5]:
for c in df.columns:
    if df[c].count() < 8000:
        df = df.drop([c], axis=1)
df.info(max_cols=300)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8449 entries, 0 to 8448
Data columns (total 23 columns):
District                            8449 non-null object
PROPERTY Address                    8449 non-null object
PROPERTY Boro-Block-Lot             8449 non-null object
PROPERTY Building Classification    8449 non-null object
PROPERTY Condo Section              8449 non-null object
PROPERTY Full Market Value          8449 non-null int64
PROPERTY Gross Income per SqFt      8449 non-null float64
PROPERTY Gross SqFt                 8449 non-null int64
PROPERTY Market Value per SqFt      8448 non-null float64
PROPERTY Neighborhood               8449 non-null object
PROPERTY Total Units                8449 non-null int64
PROPERTY Year Built                 8353 non-null float64
RENTAL 1 Address                    8448 non-null object
RENTAL 1 Boro-Block-Lot             8449 non-null object
RENTAL 1 Building Classification    8449 non-null object
RENTAL 1 Full Market Value          84

In [6]:
df=df.dropna()
df.head(5)

,District,PROPERTY Address,PROPERTY Boro-Block-Lot,PROPERTY Building Classification,PROPERTY Condo Section,PROPERTY Full Market Value,PROPERTY Gross Income per SqFt,PROPERTY Gross SqFt,PROPERTY Market Value per SqFt,PROPERTY Neighborhood,...,RENTAL 1 Boro-Block-Lot,RENTAL 1 Building Classification,RENTAL 1 Full Market Value,RENTAL 1 Gross Income per SqFt,RENTAL 1 Gross SqFt,RENTAL 1 Market Value per SqFt,RENTAL 1 Neighborhood,RENTAL 1 Total Units,RENTAL 1 Year Built,Year
0,Bronx,"717 KELLY STREET\nBronx, New York\n(40.8158575...",2-02686-7501,R2-WALK-UP,53-R1,583003,11.50,14384,40.5,MORRISANIA/LONGWOOD,...,2-02427-0001,C7-WALK-UP,395000,10.58,11055,35.7,MELROSE/CONCOURSE,16,1923,2008
1,Bronx,"1209 NELSON AVENUE\nBronx, New York\n(40.83785...",2-02516-7501,R4-ELEVATOR,73-R1,1609988,16.33,24262,66.0,HIGHBRIDGE/MORRIS HEIGHTS,...,2-02474-0051,C1-WALK-UP,1190000,16.47,17750,67.0,MELROSE/CONCOURSE,20,1910,2008
2,Bronx,"3240 NETHERLAND AVENUE\nBronx, New York\n(40.8...",2-05787-7501,R4-ELEVATOR,30-R1,760999,18.52,10108,75.0,RIVERDALE,...,2-03145-0025,C1-WALK-UP,360000,13.52,9424,38.2,FORDHAM,16,1925,2008
3,Bronx,"2718 SCHURZ AVENUE\nBronx, New York\n(40.81359...",2-05604-7501,R4-ELEVATOR,16-R1,4000024,15.50,63416,63.0,THROGS NECK,...,2-03142-0200,D1-ELEVATOR,4880000,18.25,94682,52.0,FORDHAM,99,1972,2008
4,Bronx,"720 217 STREET\nBronx, New York\n(40.881931979...",2-04664-7501,R2-WALK-UP,75-R1,1310008,16.76,19225,68.0,WILLIAMSBRIDGE,...,2-03740-0052,C1-WALK-UP,612000,9.67,18720,32.7,SOUNDVIEW,25,1926,2008


In [7]:
df.to_csv('condominium.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8351 entries, 0 to 8448
Data columns (total 23 columns):
District                            8351 non-null object
PROPERTY Address                    8351 non-null object
PROPERTY Boro-Block-Lot             8351 non-null object
PROPERTY Building Classification    8351 non-null object
PROPERTY Condo Section              8351 non-null object
PROPERTY Full Market Value          8351 non-null int64
PROPERTY Gross Income per SqFt      8351 non-null float64
PROPERTY Gross SqFt                 8351 non-null int64
PROPERTY Market Value per SqFt      8351 non-null float64
PROPERTY Neighborhood               8351 non-null object
PROPERTY Total Units                8351 non-null int64
PROPERTY Year Built                 8351 non-null float64
RENTAL 1 Address                    8351 non-null object
RENTAL 1 Boro-Block-Lot             8351 non-null object
RENTAL 1 Building Classification    8351 non-null object
RENTAL 1 Full Market Value          83

# Проверка статистических гипотез

In [8]:
df.groupby('District').mean()

,PROPERTY Full Market Value,PROPERTY Gross Income per SqFt,PROPERTY Gross SqFt,PROPERTY Market Value per SqFt,PROPERTY Total Units,PROPERTY Year Built,RENTAL 1 Full Market Value,RENTAL 1 Gross Income per SqFt,RENTAL 1 Gross SqFt,RENTAL 1 Market Value per SqFt,RENTAL 1 Total Units,RENTAL 1 Year Built,Year
District,,,,,,,,,,,,,
Bronx,8198824.894515,14.195907,240537.046414,50.763291,237.320675,1971.367089,3412864.978903,13.778945,74840.308017,44.556456,69.894515,1950.464135,2009.658228
Brooklyn,3180693.735784,18.545637,40054.782843,81.101176,76.644608,1759.053922,2719538.269118,20.194765,34228.954412,77.045828,39.215196,1942.656373,2009.711275
Manhattan,20554147.047023,34.282893,114231.795945,167.084625,93.808024,1941.921700,23942121.872304,36.664062,126587.916954,179.352552,126.105047,1951.229508,2009.625324
Queens,4151240.230999,17.740827,62911.063338,71.381446,64.119970,1932.971684,4311483.606557,17.916028,65241.362146,70.575000,71.824143,1973.789121,2009.719821
Staten_Island,3165155.750000,6.218542,80956.687500,39.809062,78.989583,1548.395833,3432447.916667,12.786354,93324.312500,40.776667,105.072917,1965.875000,2009.500000


In [9]:
df.mean()

PROPERTY Full Market Value        13123647.679679
PROPERTY Gross Income per SqFt          26.887581
PROPERTY Gross SqFt                  91066.474913
PROPERTY Market Value per SqFt         125.936677
PROPERTY Total Units                    88.746976
PROPERTY Year Built                   1892.123937
RENTAL 1 Full Market Value        14784805.420788
RENTAL 1 Gross Income per SqFt          28.704146
RENTAL 1 Gross SqFt                  92316.949467
RENTAL 1 Market Value per SqFt         131.461862
RENTAL 1 Total Units                    94.319483
RENTAL 1 Year Built                   1952.907197
Year                                  2009.660999
dtype: float64

## 1) Простая гипотеза

### 1.1) Зависит ли площадь от того, арендуется или покупается недвижимость?
$$H_0: S_{rental} \neq S_{property}$$
$$H_1: S_{rental} = S_{property}$$

In [10]:
alpha = 0.05
rent = df['RENTAL 1 Gross SqFt']
prop = df['PROPERTY Gross SqFt']
print "%s гипотеза верна" %("Нулевая" if stats.ttest_ind(prop, rent)[1] > alpha else "Альтернативная")

Нулевая гипотеза верна


### 1.2) Зависит ли площадь от района?
$$H_0: S_{district_1} \neq S_{district_2}$$
$$H_1: S_{district_1} = S_{district_2}$$

In [12]:
alpha = 0.05
rent = df['RENTAL 1 Gross SqFt']
prop = df['PROPERTY Gross SqFt']
print "%s гипотеза верна" %("Нулевая" if stats.ttest_ind(prop, rent)[1] > alpha else "Альтернативная")

Нулевая гипотеза верна


In [13]:
dstr = df["District"].unique()
m = pd.DataFrame(index=dstr, columns=dstr)
for d1 in dstr:
    for d2 in dstr:
        m[d1][d2] = "Not equal" if (stats.ttest_ind(pd.concat([rent[df.District == d1], prop[df.District == d1]], ignore_index=True),\
                                                    pd.concat([rent[df.District == d2], prop[df.District == d2]], ignore_index=True))[1] < alpha) else "Equal"
m

,Bronx,Brooklyn,Manhattan,Queens,Staten_Island
Bronx,Equal,Not equal,Not equal,Not equal,Equal
Brooklyn,Not equal,Equal,Not equal,Not equal,Not equal
Manhattan,Not equal,Not equal,Equal,Not equal,Not equal
Queens,Not equal,Not equal,Not equal,Equal,Not equal
Staten_Island,Equal,Not equal,Not equal,Not equal,Equal


## 2) Сложная гипотеза: критерий согласия 

### 2.1) Доход с квадратного фута распределен так же, как и стоимость квадратного фута

In [30]:
plt.figure(figsize=(20,10))
df['PROPERTY Market Value per SqFt'].hist(bins=100)
df['PROPERTY Gross Income per SqFt'].hist(bins=100, alpha=.7)
plt.savefig('rent.png')

In [39]:
coef = np.corrcoef(pd.concat([df['PROPERTY Market Value per SqFt'], df['RENTAL 1 Market Value per SqFt']], ignore_index=True),\
            pd.concat([df['PROPERTY Gross Income per SqFt'], df['RENTAL 1 Gross Income per SqFt']], ignore_index=True))[0][1]

In [50]:
alpha = 0.01
def test_significance(r, n, table):
    return table < math.sqrt(r * r * (n - 2) / (1 - r * r))
significance_interval = 1 - alpha
t_table = stats.t.ppf(significance_interval, len(df) * 2 - 1, loc=1)
print "Данные сильно скоррелированы" if test_significance(coef, len(df) * 2, t_table) else "Данные слабо скоррелированы"

Данные сильно скоррелированы


In [ ]:
coef = np.corrcoef(pd.concat([df['PROPERTY Market Value per SqFt'], df['RENTAL 1 Market Value per SqFt']], ignore_index=True),\
            pd.concat([df['PROPERTY Gross Income per SqFt'], df['RENTAL 1 Gross Income per SqFt']], ignore_index=True))[0][1]